# Preliminary experiment

In this setup there is an ensemble (a) which gets stimulation inputs in 6 dimensions: x, y and z axes and the Aircraft principal axes roll, pitch and yaw. The ensamble a integrates the inputs on each axis in order to calculate the correction signal which bring the system to the target state (in our first case - the origin (0, 0, 0, 0, 0, 0))

In [ ]:
# package installation
!pip install nengo
!pip install nengo-gui
!pip install numpy
!pip install matplotlib

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import nengo
from nengo.processes import Piecewise

In [ ]:
import nengo

model = nengo.Network()
with model:
    
    # create the stimuli. stimuli are separated for simplicity
    # stim_x = nengo.Node(0)
    # stim_y = nengo.Node(0)
    # stim_z = nengo.Node(0)
    # stim_roll = nengo.Node(0)
    # stim_pitch = nengo.Node(0)
    # stim_yaw = nengo.Node(0)
    stim = nengo.Node([0, 0, 0, 0, 0, 0])
    
    # create ensamble a
    a = nengo.Ensemble(n_neurons=50, dimensions=6)
    
    # connect the stimuli to ensamble a
    # nengo.Connection(stim_x, a)
    # nengo.Connection(stim_y, a)
    # nengo.Connection(stim_z, a)
    # nengo.Connection(stim_roll, a)
    # nengo.Connection(stim_pitch, a)
    # nengo.Connection(stim_yaw, a)
    nengo.Connection(stim, a)

    # create ensamble for the integration
    integration = nengo.Ensemble(n_neurons=50, dimensions=12)
    
    # create ensamble which represent the target state
    target = nengo.Ensemble(n_neurons=50, dimensions=6)
    
    # create ensamble which represent the correction signal which brings the ensamble a to the target state
    correction_signal = nengo.Ensemble(n_neurons=50, dimensions=6)
    
    # connect ensamble a to the integration ensamble (6 first dims)
    nengo.Connection(a, integration[:6])
    
    # connect the target ensamble to the integration ensamble (6 last dims)
    nengo.Connection(target, integration[6:])

    # create a function to calculate the diff between the target and the input for each axis
    def correction(inputs):
        return [
                (inputs[6] -  inputs[0]),       # x axis
                (inputs[7] -  inputs[1]),       # y axis
                (inputs[8] -  inputs[2]),       # z axis
                (inputs[9] -  inputs[3]) % 360, # roll axis
                (inputs[10] - inputs[4]) % 360, # pitch axis
                (inputs[11] - inputs[5]) % 360, # yaw axis
                ]
        pass
    
    # connect the integration ensamble to the correction signal ensamble
    nengo.Connection(integration, correction_signal, function=correction)


In [ ]:
with model:
    # Indicate which values to record
    correction_signal_probe = nengo.Probe(correction_signal)

In [ ]:
with nengo.Simulator(model) as sim:
    sim.run(5)

In [ ]:
plt.figure()
plt.plot(sim.trange(), sim.data[correction_signal_probe])
plt.xlabel('Time (s)')
#plt.legend(['$x_0$', '$x_1$', r'$\omega$']);